In [1]:
# imports
import pandas as pd
import scipy as sp 
import math
import plotly.express as px

from _mic_position_functions import calculate_distance, parametric_model, calc_error_optimized_coords

steps:
* import meaasured 3d mic positions and distance measurements
* only use points with measured distance
* calculate sum of squarred errors
* build a mic positions model with parameters (offsets and tilt)
* optimize those paramters for minimal sum of squarred errors
* return optimized paramters and mic positions

In [2]:
# read meaasured 3d mic positions
df_mic_coords = pd.read_excel("data/3d_measured_coords.xlsx", index_col=0)

# read distance measurements
df_distances = pd.read_excel("data/mic_pos_distances.xlsx")

In [3]:
# check if calculated distances match

print(df_distances.head())

print("")
for index, row in df_distances.iterrows():
    
    dist = calculate_distance(df_mic_coords, row[0], row[1], print_dist=False)

    if math.isclose(dist, df_distances["dist calculated"][index], abs_tol=0.001) == False:
        print(index)
        print("different distance")

print("all distances checked")

  Mic_Index 1 Mic_Index 2  dist measured  dist calculated  error
0         D13         D16          1.260            1.256  0.004
1         B17         C16          0.670            0.673 -0.003
2         C10         D10          1.175            1.181 -0.006
3          C2         B15          1.650            1.651 -0.001
4          A7          B6          1.416            1.397  0.019

all distances checked


In [4]:
# get all mic positions used for measurng distances (no further used)
# get indixes
measured_mic_pos = []

for mic_index_temp in df_distances["Mic_Index 1"]:
    measured_mic_pos.append(mic_index_temp)

for mic_index_temp in df_distances["Mic_Index 2"]:
    measured_mic_pos.append(mic_index_temp)

measured_mic_pos = set(measured_mic_pos) # remove double values
measured_mic_pos = sorted(measured_mic_pos) # sort values

# filter dataframe
df_mic_coords_filtered = df_mic_coords[df_mic_coords.isin(measured_mic_pos).any(axis=1)]


print(len(measured_mic_pos), "different mics used for measurng distances")
print("")
print(df_mic_coords_filtered)

25 different mics used for measurng distances

   Mic_Index  Plane      X      Y      Z
5         B6   left  1.370  0.989  0.629
7         B8   left  1.370  1.315  0.718
12       B13   left  1.370  0.157  1.257
14       B15   left  1.370  1.044  1.485
15       B16   left  1.370  1.392  1.510
16       B17   left  1.370  0.674  1.689
18        C2  front  0.913  0.028  0.267
20        C4  front  1.322  0.028  0.380
21        C5  front  0.571  0.028  0.425
26       C10  front  0.299  0.028  0.990
32       C16  front  1.313  0.028  1.508
34        D1  right  0.028  0.817  0.023
36        D3  right  0.028  1.215  0.289
38        D5  right  0.028  0.905  0.427
40        D7  right  0.028  1.331  0.677
43       D10  right  0.028  1.177  0.990
46       D13  right  0.028  1.392  1.260
49       D16  right  0.028  0.161  1.511
50       D17  right  0.028  0.877  1.690
51        A1    top  0.590  1.246  1.871
54        A4    top  1.376  0.864  1.871
55        A5    top  1.028  0.857  1.871
56        

In [5]:
# calculate sum of squarred errors

for index, row in df_distances.iterrows():
    
    dist = calculate_distance(df_mic_coords, row[0], row[1], print_dist=False)
    sq_error = (dist - df_distances["dist measured"][index])**2
    df_distances.loc[index, "squared error"] = sq_error

sum_sq_errors_default = df_distances["squared error"].sum()
print(round(sum_sq_errors_default, 5), "= default sum of squared errors")


0.00696 = default sum of squared errors


In [6]:
# Parameter

# translations = shift = offset

# Rotations:
# Roll (rotation around the plane's local X-axis)
# Pitch (rotation around the plane's local Y-axis)
# Yaw (rotation around the plane's local Z-axis) --> assumption: no yaw

# shift is in m
# roll & pitch in degrees

# top plane
A_x_shift = 0
A_y_shift = 0

# left plane
# B_x_shift = 0 # keep 0 for reference
B_y_shift = 0
B_z_shift = 0
B_roll = 0 # rotation over 3d y axis 
B_pitch = 0 # rotation over 3d z axis

# front plane
C_x_shift = 0
# C_y_shift = 0 # keep 0 for reference
C_z_shift = 0
C_roll = 0 # rotation over 3d x axis 
C_pitch = 0 # rotation over 3d z axis

# right_plane
D_x_shift = 0
D_y_shift = 0
D_z_shift = 0
D_roll = 0 # rotation over 3d y axis 
D_pitch = 0 # rotation over 3d z axis


initial_parameters = [
    A_x_shift,
    A_y_shift,
    B_y_shift,
    B_z_shift,
    B_roll,
    B_pitch,
    C_x_shift,
    C_z_shift,
    C_roll,
    C_pitch,
    D_x_shift,
    D_y_shift,
    D_z_shift,
    D_roll,
    D_pitch
    ]


print(len(initial_parameters), "parameter in total")
print(initial_parameters)

15 parameter in total
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:

def optimizer_wrapper(initial_parameters, df_mic_coords=df_mic_coords, df_distances=df_distances):
    df_mic_coords_moved = parametric_model(df_mic_coords, initial_parameters)
    sum_sq_errors, _ = calc_error_optimized_coords(df_distances, df_mic_coords_moved)

    return sum_sq_errors

In [8]:
result = sp.optimize.minimize(optimizer_wrapper, initial_parameters)

print("Optimal variables:", result.x)
print("Optimal function value:", result.fun)

Optimal variables: [-0.07855106 -0.04049084  0.01434719  0.0119259  -0.71233223 -0.67175506
  0.03950037 -0.00558461  1.0641494   1.60576729 -0.04041927 -0.03496487
 -0.02147978 -0.16139158 -0.38482831]
Optimal function value: 1.34779251190118e-05


In [9]:
df_mic_coords_optimized = parametric_model(df_mic_coords, result.x)

# Create the first 3D scatter plot
fig = px.scatter_3d(df_mic_coords, 
                    x='X', 
                    y='Y', 
                    z='Z', 
                    text='Mic_Index', 
                    title='3D Scatter Plot', 
                    color='Plane', 
                    width=1000, 
                    height=1000)

# Create the second 3D scatter plot (optimized data)
fig_optimized = px.scatter_3d(df_mic_coords_optimized, 
                               x='X_optimized', 
                               y='Y_optimized', 
                               z='Z_optimized', 
                               text='Mic_Index', 
                               color='Plane')

# Customize the color of the second plot to differentiate it
for trace in fig_optimized.data:
    trace.marker.color = 'rgba(0, 0, 0, 0.7)'  # black with some transparency
    trace.name = 'Optimized Data'  # Label in the legend

# Add each trace from the second figure to the first
for trace in fig_optimized.data:
    fig.add_trace(trace)

# Display the combined figure
fig.show()
